In [1]:
import pandas as pd
from tqdm import tqdm
import os
if not os.path.exists('data/processed/'):
    os.makedirs('data/processed/')

In [2]:
tqdm.pandas()

In [3]:
SOURCES = ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 
           'JiraEcosystem', 'MariaDB', 'Mojang', 'MongoDB', 
           'Qt', 'RedHat', 'Sakai', 'SecondLife', 'Sonatype', 'Spring']

In [5]:
def load_data(source):
    #Loading Issues
    filename = '../data/raw/issues_'+source.lower()+'.csv'
    issue_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=';', index_col=['issue_id'])

    #Loading Links
    filename = '../data/raw/links_'+source.lower()+'.csv'
    link_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=';').drop_duplicates()
    
    return issue_df, link_df

In [4]:
# this function is to identify doubled issue pairs
def add_linked_issues_to_df(df):
    df['issues']=''
    for i in tqdm(range(len(df))):
        df["issues"].iloc[i] = str(sorted(set([df.iloc[i]['issue_id_1'], df.iloc[i]['issue_id_2']])))

In [6]:
def clean_issues(issue_df):
    #Remove issues with empty titles
    issue_df = issue_df[~issue_df['title'].fillna(' ').str.isspace()]
    print(f'After filtering out issues with empty titles, {len(issue_df)} issues remain')
    
    return issue_df

In [7]:
def clean_links(link_df):    
    # remove links with uncrawled and filtered issues
    link_df = link_df[link_df[['issue_id_1', 'issue_id_2']].isin(issue_df.index.values).all(axis=1)]
    print(f'Left with {len(link_df)} links after removing half-private links')
    
    # cleanup links
    # only allow one linktype per issue-pair
    link_df.drop_duplicates(subset=['name'], keep=False, inplace=True)
    print(f'Left with {len(link_df)} links after removing issue-pairs with multiple links between them')

    # in case the name is the otherway around, like issue-1_issue-2 and issue-2_issue-1
    doublelinks = (link_df.issues.value_counts()>1).rename_axis('doubles').reset_index(name='valid')
    valid_doubles = set(doublelinks[doublelinks['valid']==True]['doubles'])

    for i in tqdm(valid_doubles):
        if len(set(link_df[link_df['issues']==i]['linktype']))>1:
            link_df = link_df[link_df.issues != i]
    print(f'Left with {len(link_df)} links after removing issue-pairs with multiple link types between them')

    #Multiple links complete remove
    link_df.drop_duplicates(subset=['issues'], inplace=True)
    print(f'Left with {len(link_df)} links after removing issue-pairs with multiple entries')

    link_df.reset_index(inplace=True, drop=True)
    
    return link_df

In [8]:
# Creatres non-links from randomly selected issues.
import random
def create_non_links(issue_df, link_df, linked_pairs):
   
    non_link_issues = set(issue_df[~issue_df['resolution'].isin(['Duplicate'])].index)
    no_issues = len(non_link_issues)
    
    cols = ['name', 'linktype', 'issue_id_1', 'issue_id_2', 'issues']
    non_links_df = pd.DataFrame(columns = cols)

    for i in tqdm(range(int(link_df.linktype.value_counts().mean()))):
        sample = random.sample(non_link_issues, 2)
        if not set([sample[0], sample[1]]) in (linked_pairs):
            name = str(sample[0]) + "_" + str(sample[1])
            non_links_df = non_links_df.append({
                "name": name,
                "linktype": "Non-Link",
                "issue_id_1":  sample[0],
                "issue_id_2":  sample[1],
                "issues": str(sorted(set([sample[0], sample[1]])))
            }, ignore_index=True)
        else:
            print('hi')
        
    link_plus_df = link_df.append(non_links_df, ignore_index=True)
    
    return link_plus_df

In [18]:
for s in SOURCES:
    print(s.upper())
    issue_df, link_df = load_data(s)
    issue_df.to_csv('data/processed/issues_'+s.lower()+'.csv', encoding='utf-8', index=True, sep=';')
    
    print(f'Loaded {len(issue_df)} issues and {len(link_df)} links')
    
    issue_df = clean_issues(issue_df)
    
    add_linked_issues_to_df(link_df)
    
    linked_pairs = set(link_df['issues'])
    
    link_df = clean_links(link_df)
    link_plus_df = create_non_links(issue_df, link_df, linked_pairs)
    
    print(f'Cleaned {len(issue_df)} issues and {len(link_df)} links')
    
    link_df.to_csv('data/processed/links_'+s.lower()+'.csv', encoding='utf-8', index=True, sep=';')
    
    print(link_df.linktype.value_counts())
    
    print(f'Created link and non_links of size {len(link_plus_df)}')
    
    link_plus_df.to_csv('data/processed/links_plus_'+s.lower()+'.csv', encoding='utf-8', index=True, sep=';')
    print("----------------------------")
    print()

APACHE
Loaded 1014926 issues and 264107 links


  0%|          | 172/264107 [00:00<05:10, 850.84it/s]

After filtering out issues with empty titles, 1014925 issues remain


100%|██████████| 264107/264107 [06:09<00:00, 715.57it/s]
<ipython-input-7-d8abf72b2809>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link_df.drop_duplicates(subset=['name'], keep=False, inplace=True)
  0%|          | 0/2218 [00:00<?, ?it/s]

Left with 263647 links after removing half-private links
Left with 259717 links after removing issue-pairs with multiple links between them


...

Loaded 69156 issues and 14716 links
After filtering out issues with empty titles, 69156 issues remain


100%|██████████| 14716/14716 [00:07<00:00, 1930.40it/s]
<ipython-input-7-d8abf72b2809>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link_df.drop_duplicates(subset=['name'], keep=False, inplace=True)
  2%|▏         | 38/1606 [00:00<00:04, 372.53it/s]

Left with 14616 links after removing half-private links
Left with 14550 links after removing issue-pairs with multiple links between them
Left with 14478 links after removing issue-pairs with multiple link types between them
Left with 14462 links after removing issue-pairs with multiple entries


100%|██████████| 1606/1606 [00:04<00:00, 354.80it/s]


Cleaned 69156 issues and 14462 links
Relate           5909
Subtask          1941
Duplicate        1745
Epic-Relation    1635
Depend           1259
Related           993
Depends           488
Supersede         478
Cloners            14
Name: linktype, dtype: int64
Created link and non_links of size 16068
----------------------------

